In [1]:
from flask import Flask , request , jsonify
import pyodbc
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
import threading
import time
from datetime import datetime
import requests
import json

In [2]:
app = Flask(__name__)


In [3]:
def connect_db():
    conn_str = (
        'DRIVER={ODBC Driver 17 for SQL Server};'
        'SERVER=localhost\\SQLEXPRESS;'
        'DATABASE=SalesDB;'
        'Trusted_Connection=yes;'
    )
    return pyodbc.connect(conn_str)

In [ ]:
@app.route("/customers", methods=["POST"])
def insert_customer():
    data = request.get_json()
    try:
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO Customers (Name, Gender, BirthDate, Phone, Email, City)
            VALUES (?, ?, ?, ?, ?, ?)
        """, (
            data['Name'],
            data['Gender'],
            data.get('BirthDate'),
            data.get('Phone'),
            data.get('Email'),
            data.get('City')
        ))
        conn.commit()
        cursor.close()
        conn.close()
        return jsonify({"message": "Customer inserted successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route("/branches", methods=["POST"])
def insert_branch():
    data = request.get_json()
    try:
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO Branches (BranchName, City, Address)
            VALUES (?, ?, ?)
        """, (
            data['BranchName'],
            data.get('City'),
            data['Address']
        ))
        conn.commit()
        cursor.close()
        conn.close()
        return jsonify({"message": "Branch inserted successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route('/products', methods=['POST'])
def insert_product():
    data = request.get_json()
    try:
        conn = connect_db()
        cursor = conn.cursor()
        cursor.execute("""
            INSERT INTO Products (ProductName, Category, Price, Supplier)
            VALUES (?, ?, ?, ?)
        """, (
            data['ProductName'],
            data.get('Category'),
            data.get('Price'),
            data.get('Supplier')
        ))
        conn.commit()
        cursor.close()
        conn.close()
        return jsonify({"message": "Product inserted successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route('/sales', methods=['POST'])
def insert_sale():
    data = request.get_json()
    try:
        conn = connect_db()
        cursor = conn.cursor()

        cursor.execute("""
            INSERT INTO Sales (CustomerID, BranchID, SaleDate, TotalAmount)
            VALUES (?, ?, ?, ?)
        """, (
            data['CustomerID'],
            data['BranchID'],
            data.get('SaleDate', datetime.now().date()),
            data['TotalAmount']
        ))

        conn.commit()

        cursor.close()
        conn.close()
        return jsonify({"message": "Sale inserted successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500


@app.route("/sale_details", methods=["POST"])
def insert_sale_detail():
    data = request.get_json()
    try:
        conn = connect_db()
        cursor = conn.cursor()

        cursor.execute("""
            INSERT INTO SaleDetails (SaleID, ProductID, Quantity, UnitPrice, TotalPrice)
            VALUES (?, ?, ?, ?, ?)
        """, (
            data['SaleID'],
            data['ProductID'],
            data['Quantity'],
            data['UnitPrice'],
            data['TotalPrice']
        ))

        conn.commit()

        cursor.close()
        conn.close()
        return jsonify({"message": "Sale detail inserted successfully"})
    except Exception as e:
        return jsonify({"error": str(e)}), 500

    
def run_flask():
    app.run(port=5000 , debug=False , use_reloader=False)

# code to run the flask API in a separate thread in the background
threading.Thread(target=run_flask).start()
time.sleep(1)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_details HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_details HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_details HTTP/1.1" 500 -
127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_details HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_details HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_details HTTP/1.1" 500 -
127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_details HTTP/1.1" 500 -
127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_details HTTP/1.1" 500 -
127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_details HTTP/1.1" 500 -
127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_details HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_details HTTP/1.1" 500 -
127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_details HTTP/1.1" 200 -
127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_details HTTP/1.1" 500 -
127.0.0.1 - - [27/Aug/2025 23:53:31] "POST /sale_de

In [ ]:
endpoints = {
    "customers.json": "http://127.0.0.1:5000/customers",
    "branches.json": "http://127.0.0.1:5000/branches",
    "products.json": "http://127.0.0.1:5000/products",
    "sales.json": "http://127.0.0.1:5000/sales",
    "sale_details.json": "http://127.0.0.1:5000/sale_details"
}

# Send each JSON file to its corresponding API endpoint
for file_name, url in endpoints.items():
    try:
        with open(file_name, "r", encoding="utf-8") as f:
            data_list = json.load(f)  # Must be a list of dicts

        for record in data_list:
            res = requests.post(url, json=record)
            print(f"POST to {url} -> {res.status_code} {res.text}")

    except FileNotFoundError:
        print(f"❌ File {file_name} not found!")
    except json.JSONDecodeError:
        print(f"❌ Invalid JSON format in {file_name}")


POST to http://127.0.0.1:5000/sale_details -> 200 {"message":"Sale detail inserted successfully"}

POST to http://127.0.0.1:5000/sale_details -> 200 {"message":"Sale detail inserted successfully"}

POST to http://127.0.0.1:5000/sale_details -> 500 {"error":"('23000', '[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The INSERT statement conflicted with the FOREIGN KEY constraint \"FK__SaleDetai__SaleI__4316F928\". The conflict occurred in database \"SalesDB\", table \"dbo.Sales\", column \\'SaleID\\'. (547) (SQLExecDirectW); [23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The statement has been terminated. (3621)')"}

POST to http://127.0.0.1:5000/sale_details -> 200 {"message":"Sale detail inserted successfully"}

POST to http://127.0.0.1:5000/sale_details -> 200 {"message":"Sale detail inserted successfully"}

POST to http://127.0.0.1:5000/sale_details -> 500 {"error":"('23000', '[23000] [Microsoft][ODBC Driver 17 for SQL Server][SQL Server]The INSERT state